In [1]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

## S4 H2 adsorption isotherm, lumped injections

### Reading in the data and preparing DataFrames

In [2]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
0,NaN,2023-02-23 12:38:12,0.0,88.984,7.790000e-10,5.720000e-10,4.996000e-09,3.661000e-07,1.1,456.0,-5.275,4270.802,NaN,NaN
1,"inj volume prepared, baseline mass scan done",2023-02-23 12:38:19,7.0,88.985,7.730000e-10,5.710000e-10,4.996000e-09,3.656000e-07,1.1,456.0,-5.275,4271.027,NaN,NaN
2,NaN,2023-02-23 12:38:26,14.0,88.984,7.800000e-10,5.710000e-10,4.996000e-09,3.670000e-07,1.1,456.0,-5.275,4270.914,NaN,NaN
3,NaN,2023-02-23 12:38:33,21.0,88.985,7.790000e-10,5.710000e-10,4.996000e-09,3.670000e-07,1.1,456.0,-5.275,4270.802,NaN,NaN
4,NaN,2023-02-23 12:38:40,28.0,88.985,7.900000e-10,5.750000e-10,4.996000e-09,3.665000e-07,1.1,456.0,-5.275,4270.914,NaN,NaN


### Data processing

In [3]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'inj volume prepared, baseline mass scan done' 'RGA filament off'
 'turbo off' 'bp check']


#### CernOx temperature R-T conversion

In [4]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Ignore end of measurement

In [5]:
#slicing the dataset
ramp = vaclog.iloc[list((vaclog["Elapsed time"]/3600).between(0,2))]
print(ramp)

                                     Live comments                Time  \
0                                              NaN 2023-02-23 12:38:12   
1     inj volume prepared, baseline mass scan done 2023-02-23 12:38:19   
2                                              NaN 2023-02-23 12:38:26   
3                                              NaN 2023-02-23 12:38:33   
4                                              NaN 2023-02-23 12:38:40   
...                                            ...                 ...   
1050                                           NaN 2023-02-23 14:37:44   
1051                                           NaN 2023-02-23 14:37:51   
1052                                           NaN 2023-02-23 14:37:58   
1053                                           NaN 2023-02-23 14:38:05   
1054                                           NaN 2023-02-23 14:38:11   

      Elapsed time  injection 100mbar      Barion_2      Barion_1  \
0              0.0             88.984  7.7

## Subtracting baseline

In [6]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = vaclog.loc[vaclog["Barion_2"]/0.46 >= 9e-8]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[633]).dt.total_seconds()    
inj.insert(1,"Elapsed time",runtime2)



     Live comments                Time  injection 100mbar      Barion_2  \
92             NaN 2023-02-23 12:48:48             88.986  1.800000e-07   
93             NaN 2023-02-23 12:48:55             88.985  1.300000e-07   
94             NaN 2023-02-23 12:49:02             88.985  1.890000e-07   
95             NaN 2023-02-23 12:49:09             88.985  2.670000e-07   
96             NaN 2023-02-23 12:49:16             88.981  3.110000e-07   
...            ...                 ...                ...           ...   
2042           NaN 2023-02-23 16:31:35             83.563  2.660000e-06   
2043           NaN 2023-02-23 16:31:41             83.562  2.660000e-06   
2044           NaN 2023-02-23 16:31:49             83.563  2.660000e-06   
2045           NaN 2023-02-23 16:31:55             83.563  2.660000e-06   
2046           NaN 2023-02-23 16:32:02             83.566  2.660000e-06   

          Barion_1  DUAL experiment  DUAL insulation  injection 1mbar  helium  \
92    1.830000e-07

C:\Users\etiirine\AppData\Local\Temp\ipykernel_8284\522004775.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


In [7]:
#TODO - subtract the baseline from graph 2 (linear pressure scale) and plot again
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [8]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [9]:
print(p_coef)
inj.head()

0.11972665112564158


,Live comments,Elapsed time,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
92,NaN,-3692.0,2023-02-23 12:48:48,4.684956e-08,4.763039e-08,3.913043e-07,3.978261e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,-3685.0,2023-02-23 12:48:55,3.383579e-08,3.539744e-08,2.826087e-07,2.956522e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,-3678.0,2023-02-23 12:49:02,4.919204e-08,5.205507e-08,4.108696e-07,4.347826e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,-3671.0,2023-02-23 12:49:09,6.949351e-08,7.183599e-08,5.804348e-07,6.000000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
96,NaN,-3664.0,2023-02-23 12:49:16,8.094563e-08,8.927444e-08,6.760870e-07,7.456522e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN


In [10]:
#define baseline
base1 = inj[inj["corr BA1"].between(4.6e-6,6e-6)]
base2 = inj[inj["corr BA2"].between(4.6e-6,6e-6)]

#take the average of the baseline and apply thermal transpiration coef to these values
avg2 = np.mean(base1["corr BA2"])*p_coef
avg1 = np.mean(base2["corr BA1"])*p_coef
print(avg1)
print(avg2)


6.736526276196968e-07
6.82541479830431e-07


In [11]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot(inj["Time"],inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot(inj["Time"],inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


Text(0.5, 1.0, 'Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ')

### Injection pressure linearity

In [12]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.58*x+84.49


### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [13]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


0.3423746161720071
[0.00000000e+00 1.61126774e+15 1.61126774e+15 ... 1.07745474e+19
 1.07745474e+19 1.07793812e+19]
[0.00000000e+00 5.83792661e+12 5.83792661e+12 ... 3.90382153e+16
 3.90382153e+16 3.90557290e+16]


In [14]:
inj.insert(2, "Coverage",N_cov)


In [15]:
inj.head()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
92,NaN,-3692.0,0.000000e+00,2023-02-23 12:48:48,4.684956e-08,4.763039e-08,3.913043e-07,3.978261e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,-3685.0,5.837927e+12,2023-02-23 12:48:55,3.383579e-08,3.539744e-08,2.826087e-07,2.956522e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,-3678.0,5.837927e+12,2023-02-23 12:49:02,4.919204e-08,5.205507e-08,4.108696e-07,4.347826e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,-3671.0,5.837927e+12,2023-02-23 12:49:09,6.949351e-08,7.183599e-08,5.804348e-07,6.000000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
96,NaN,-3664.0,2.918963e+13,2023-02-23 12:49:16,8.094563e-08,8.927444e-08,6.760870e-07,7.456522e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN


In [16]:
## dropping coverage values close to 0
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
96,NaN,-3664.0,2.918963e+13,2023-02-23 12:49:16,8.094563e-08,8.927444e-08,6.760870e-07,7.456522e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN
97,NaN,-3658.0,2.918963e+13,2023-02-23 12:49:22,1.345623e-07,1.361240e-07,1.123913e-06,1.136957e-06,88.981,5.170000e-07,5.230000e-07,4.458000e-07,3.714000e-07,1.1,450.0,-5.387,4273.608,NaN,NaN
98,NaN,-3651.0,7.589305e+13,2023-02-23 12:49:29,1.553844e-07,1.579871e-07,1.297826e-06,1.319565e-06,88.973,5.970000e-07,6.070000e-07,5.032000e-07,3.718000e-07,1.1,450.0,-5.275,4273.608,NaN,NaN
99,NaN,-3644.0,1.225965e+14,2023-02-23 12:49:36,2.311245e-07,2.732891e-07,1.930435e-06,2.282609e-06,88.965,8.880000e-07,1.050000e-06,1.432000e-06,3.718000e-07,1.1,451.0,-5.163,4273.608,NaN,NaN
100,NaN,-3637.0,2.160033e+14,2023-02-23 12:49:43,5.335644e-07,5.413727e-07,4.456522e-06,4.521739e-06,88.949,2.050000e-06,2.080000e-06,1.600000e-06,3.718000e-07,1.1,450.0,-5.275,4273.720,NaN,NaN


# Plotting

In [17]:
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }

In [18]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots(figsize=(12,6))

l5 = ax.plot(vaclog["Time"], polyfit(A,vaclog["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Timestamp')
ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
ax2=ax.twinx()
l1 = ax2.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
l2 = ax2.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
l3 = ax2.plot(vaclog["Time"],vaclog['DUAL experiment'],marker=".", markersize=4,label='Dual gauge')
l4 = ax2.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')

plt.text(14.8, 7.36e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(21, 1e-5, '~Saturated vapor pressure', fontdict = font)
ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper right")
plt.title("Pressure evolution + temperatures")
ax2.set_yscale('log')

plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_overview.png')
plt.show()


In [19]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')
plt.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 CU ESD sample out - Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./S4 a-C H2 adsorption isotherm_Pressures.png')
plt.show()

In [20]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(ramp["Time"],ramp['Barion_1'],marker=".", markersize=4,label='BA1')
ax.plot(ramp["Time"],ramp['Barion_2'],marker=".", markersize=4,label='BA2')
ax.plot(ramp["Time"],ramp['DUAL experiment'],marker=".", markersize=4,label='DUAL gauge')

plt.legend(loc="upper right")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

plt.text(14.25, 6e-6, 'Pressure rise due to LHe refill \u2192 ', fontdict = font)
plt.text(13.02, 4.32e-6, 'Temperature fluctuation \u2192', fontdict = font)
plt.legend(loc="upper right")
plt.title("Pressure evolution during injection. Corrected for H2")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./S4 a-C H2 adsorption isotherm_Sliced df Pressures.png')
plt.show()


In [21]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots(figsize=(12,6))
y_fit = func(x,a,b)
ax.plot(inj["Time"],inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(12))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

In [22]:
#plot coverage vs time
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(inj["Time"],inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')

plt.text(15.30, 8e+15, 'Slope change coherent with inj. pressure graph \u2192 ', fontdict = font)
plt.title("Coverage over time")

Text(0.5, 1.0, 'Coverage over time')

In [23]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
xrange = np.arange(min(inj_new["Coverage"]),max(inj_new["Coverage"]),5e+15)
print(xrange)
ax.set_xticks(xrange)
#ax.set_xlim(left = 3e+15,right = 9.20e15)
#ax.set_xlim(right= 9.23e+15)
#ax.set_xlim(2.9e+12,9.23e+15)
#ax.set_ylim(1e-10,1.5e-6)


ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')

plt.title("H2 adsorption isotherm")

[2.91896331e+13 5.02918963e+15 1.00291896e+16 1.50291896e+16
 2.00291896e+16 2.50291896e+16 3.00291896e+16 3.50291896e+16]


Text(0.5, 1.0, 'H2 adsorption isotherm')